In [ ]:
pip install tensorflow opencv-python
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2


In [ ]:
# Load the MobileNetV2 model, excluding the top dense layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of it
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Output layer for bounding box regression (x, y, width, height)
predictions = Dense(4, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Assume your data is in directories structured as: data/train and data/validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='input'  # Adjust based on your needs; here we're using 'input' for the sake of bounding box regression
)


In [ ]:
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=100,  # Adjust based on your data size
    validation_data=None  # Add validation data if available
)


In [ ]:
# Unfreeze some layers of the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile the model again with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='mean_squared_error')

# Continue training
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=100
)


In [ ]:
def predict_and_draw_boxes(image, model):
    # Preprocess the input image
    input_image = cv2.resize(image, (128, 128))
    input_image = input_image.astype('float32') / 255.0
    input_image = np.expand_dims(input_image, axis=0)

    # Predict the bounding box
    bbox = model.predict(input_image)[0]

    # Denormalize the bounding box coordinates
    h, w, _ = image.shape
    startX, startY, width, height = bbox
    startX = int(startX * w)
    startY = int(startY * h)
    endX = int(startX + width * w)
    endY = int(startY + height * h)

    # Draw the bounding box on the image
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

    return image

# Test the model with a new image
image = cv2.imread('test_image.jpg')
output_image = predict_and_draw_boxes(image, model)

# Display the result
cv2.imshow("Detected Objects", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
